In [36]:
from __future__ import annotations

import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Optional


# =============================================================================
# Config
# =============================================================================

@dataclass(frozen=True)
class Paths:
    results_dir: Path = Path("results")
    gold_valuation: Path = Path("results/gold_valuation.xlsx")
    swap: Path = Path("results/domestic_banks_fx_swap.xlsx")
    net_reserves: Path = Path("results/net_rezervler.xlsx")
    public_fx: Path = Path("results/kamu_fx.xlsx")

    def ensure_dirs(self) -> None:
        self.results_dir.mkdir(parents=True, exist_ok=True)


# =============================================================================
# Plot Badge / Watermark
# =============================================================================

def watermark_badge(
    fig: plt.Figure,
    text: str = "econdatametrics",
    angle: float = 45,
    alpha: float = 0.10,
    fontsize: int = 52,
    color: str = "grey",
) -> None:
    
    fig.text(
        0.5, 0.5, text,
        ha="center", va="center",
        rotation=angle,
        fontsize=fontsize,
        color=color,
        alpha=alpha,
        zorder=0,
    )


# =============================================================================
# IO + Data Pipeline
# =============================================================================

def read_xlsx(path: Path) -> pd.DataFrame:
    return pd.read_excel(path)


def load_sources(paths: Paths) -> Dict[str, pd.DataFrame]:
    """Kaynakları tek yerden yükle."""
    return {
        "gold_valuation": read_xlsx(paths.gold_valuation),
        "swap": read_xlsx(paths.swap),
        "net_reserves": read_xlsx(paths.net_reserves),
        "public_fx": read_xlsx(paths.public_fx),
    }


def merge_sources(src: Dict[str, pd.DataFrame], on: str = "Tarih") -> pd.DataFrame:
    """Tüm kaynakları Tarih üzerinden inner join ile birleştir."""
    df = (
        src["gold_valuation"]
        .merge(src["swap"], on=on, how="inner")
        .merge(src["net_reserves"], on=on, how="inner")
        .merge(src["public_fx"], on=on, how="inner")
    )
    df[on] = pd.to_datetime(df[on], errors="coerce")
    return df.dropna(subset=[on]).sort_values(on)


def compute_components(df: pd.DataFrame) -> pd.DataFrame:
    """
    Bileşenleri hesapla ve tek tabloda döndür.
    Beklenen kolonlar (sende var):
    - gold_valuation
    - swap
    - net_rezerv_usd_mn
    - Kamu Mevduatı
    - usdtry_daily
    """
    out = df.copy()

    # Kamu mevduatı USD mn (yaklaşımın)
    out["Kamu Mevduatı (USD mn)"] = out["Kamu Mevduatı"] / out["usdtry_daily"] / 1_000_000

    # Δ hesapları
    out["Net Rezerv Değişimi"] = out["net_rezerv_usd_mn"].diff()
    out["Kamu Mevduatı Değişimi"] = out["Kamu Mevduatı (USD mn)"].diff()
    out["Yurt içi Banka Swap Değişimi"] = out["swap"].diff() / 1000

    # seçilecek bileşenler
    out = out.set_index("Tarih")[
        ["gold_valuation", "Net Rezerv Değişimi", "Kamu Mevduatı Değişimi", "Yurt içi Banka Swap Değişimi"]
    ].rename(columns={"gold_valuation": "Altın Değerleme"})

    # FX alış/satış residual
    out["FX Alış/Satış"] = (
        out["Net Rezerv Değişimi"]
        - out["Altın Değerleme"]
        - out["Kamu Mevduatı Değişimi"]
        - out["Yurt içi Banka Swap Değişimi"]
    )

    return out


def select_snapshot(df: pd.DataFrame, which: str = "last", date: Optional[str] = None) -> pd.DataFrame:
    """
    which:
      - "last": son tarihi alır
      - "date": belirli bir tarihi alır (date="YYYY-MM-DD")
    """
    if which == "last":
        return df.loc[[df.index.max()]]
    if which == "date":
        if date is None:
            raise ValueError("which='date' için date='YYYY-MM-DD' gerekli.")
        ts = pd.to_datetime(date)
        return df.loc[[ts]]
    raise ValueError("which must be 'last' or 'date'.")


# =============================================================================
# Plotting
# =============================================================================

def plot_snapshot_bar(
    snap: pd.DataFrame,
    title_prefix: str = "TCMB Net Rezerv Bileşenlerinin Değişimi",
    ylabel: str = "Katkı (USD mn)",
    badge_text: str = "econdatametrics",
    out_path: Optional[Path] = None,
    dpi: int = 200,
) -> Path:
    
    date_label = snap.index[0].date()
    row = snap.iloc[0]

    fig, ax = plt.subplots(figsize=(11, 5))

    bars = ax.bar(row.index, row.values)
    ax.axhline(0, linewidth=1)

    ax.set_title(f"{title_prefix} ({date_label}) — Tahmin", fontsize=13, pad=12)
    ax.set_ylabel(ylabel)
    plt.xticks(rotation=30, ha="right")

    # bar üstü değer
    for bar in bars:
        h = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            h,
            f"{h:.2f}",
            ha="center",
            va="bottom" if h >= 0 else "top",
            fontsize=9,
        )

    # gömülü diagonal badge
    watermark_badge(fig, text=badge_text)

    plt.tight_layout()

    # çıktı
    if out_path is None:
        out_path = Path("results") / "net_reserves_change_composition.png"
    out_path.parent.mkdir(parents=True, exist_ok=True)

    fig.savefig(out_path, dpi=dpi, bbox_inches="tight")
    plt.close(fig)
    return out_path


# =============================================================================
# çalıştır
# =============================================================================

def run_pipeline(
    paths: Paths = Paths(),
    which: str = "last",
    date: Optional[str] = None,
    verbose: bool = True,
) -> Tuple[pd.DataFrame, Path]:
    """
    Returns:
      - full components df (time series)
      - saved png path
    """
    paths.ensure_dirs()

    src = load_sources(paths)
    merged = merge_sources(src)
    components = compute_components(merged)

    snap = select_snapshot(components, which=which, date=date)
    png_path = plot_snapshot_bar(snap, out_path=paths.results_dir / "net_reserves_change_composition.png")

    if verbose:
        print("Saved:", png_path)

    return components, png_path


# =============================================================================
# Kullanım
# =============================================================================
if __name__ == "__main__":
    components, png_path = run_pipeline(which="last", verbose=True)
    


Saved: results/net_reserves_change_composition.png
